<a href="https://colab.research.google.com/github/HwangHanJae/Book-Recommendation/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA

데이터를 확인하고 전처리 과정을 진행하겠습니다.

# 데이터 로드

In [145]:
import pandas as pd
import numpy as np

user_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/Users.csv'
book_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/Books.csv'
ratings_path = '/content/drive/MyDrive/Colab Notebooks/Recommendation/Book Recommendation/dataset/Ratings.csv'

users = pd.read_csv(user_path)
books = pd.read_csv(book_path)
ratings = pd.read_csv(ratings_path)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 데이터 확인

### Users

In [146]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [147]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [148]:
users['User-ID'].nunique()

278858

Users 데이터의 `User-ID`(사용자ID)는 결측치(NULL)는 없고 총 278858개가 있는 것을 확인할 수 있습니다.

### Books

In [149]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [150]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [151]:
books['ISBN'].nunique()

271360

Books 데이터의 `ISBN`(국제표준도서번호)는 결측치(NULL)는 없고 271360개가 있는 것을 확인할 수 있습니다.

### Ratings

In [152]:
ratings.shape

(1149780, 3)

In [153]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [154]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

ratings 데이터에는 결측치가 없는 것으로 확인됩니다.

In [155]:
ratings['User-ID'].nunique()

105283

In [156]:
ratings['ISBN'].nunique()

340556

`nunique`는 각 컬럼의 고유값의 수를 말하는데

ratings 데이터에서 `User-ID`의 `nunique`와 Users 데이터의 `User-ID`의 `nunique`가 다른것으로 확인됩니다.

또한 ratings 데이터의 `ISBN`의 `nuinque`  books의 `ISBN`의 `nunique` 다른것으로 확인됩니다.

실제로 평점을 부여한 사용자와 평점이 부여된 아이템(책)만 가지고 추천을 이용하는 것이 좋겠습니다.

먼저 ratings에서 평점이 0인 데이터를 제거하겠습니다.

In [230]:
drop_index = ratings[ratings['Book-Rating'] == 0].index

new_ratings = ratings.drop(drop_index).reset_index(drop=True)

print(new_ratings.shape)

(433671, 3)


실제로 정보가 있고 평점이 부여한 사용자와 평점이 부여된 아이템만 추출하겠습니다.

In [231]:
unique_book_id = books['ISBN'].unique()
unique_user_id = users['User-ID'].unique()

new_ratings = new_ratings.loc[new_ratings['ISBN'].isin(unique_book_id) & new_ratings['User-ID'].isin(unique_user_id)]
new_ratings = new_ratings.reset_index(drop=False)

new_ratings.head()

,index,User-ID,ISBN,Book-Rating
0,0,276726,0155061224,5
1,1,276729,052165615X,3
2,2,276729,0521795028,6
3,5,276744,038550120X,7
4,7,276747,0060517794,9


In [243]:
rating_users = users.loc[users['User-ID'].isin(new_ratings['User-ID'].unique())]
print(rating_users.shape)
print(rating_users['User-ID'].nunique())
rating_users.head()

(68091, 3)
68091


,User-ID,Location,Age
7,8,"timmins, ontario, canada",NaN
8,9,"germantown, tennessee, usa",NaN
11,12,"fort bragg, california, usa",NaN
13,14,"mediapolis, iowa, usa",NaN
15,16,"albuquerque, new mexico, usa",NaN


In [245]:
rating_books = books.loc[books['ISBN'].isin(new_ratings['ISBN'].unique())]
print(rating_books.shape)
print(rating_books['ISBN'].nunique())
rating_books.head()

(149836, 8)
149836


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...


In [ ]:
new_ratings

ratings에는 약 34만개의 책이 존재하지만 rating_books에는 약 14만개의 책이 존재하는 것을 확인할 수 있습니다. 실제로 책에 평점이 부여되었지만 해당 책의 정보를 볼 수 없을 수 있기 때문에 책의 정보가 존재하는 아이템만 추천에 이용할 것이기 때문에 데이터가 줄어든 것입니다. 사용자(User)또한 마찬가지로 사용자의 정보가 존재하는 아이템만 추천에 이용하겠습니다.

# 데이터 저장

In [246]:
new_ratings.to_csv('New_Ratings.csv', index=False)

In [247]:
rating_users=rating_users.reset_index(drop=True)
rating_users.to_csv('Rating_Users.csv', index=False)

In [248]:
rating_books = rating_books.reset_index(drop=True)
rating_books.to_csv('Rating_Books.csv', index=False)